# This Looks Like That There

Main training notebook.

In [1]:
import os
import sys
import time
import imp #imp.reload(module)

import numpy as np
from tqdm import trange
from icecream import ic

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import tensorflow as tf

import network
import experiment_settings 
import data_functions
import push_prototypes
import plots
import common_functions

In [2]:
__author__ = "Elizabeth A. Barnes and Randal J Barnes"
__version__ = "1 December 2021"

In [3]:
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi']= 150
dpiFig = 300.

## Print the detailed system info

In [4]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"tensorflow version = {tf.__version__}")

python version = 3.9.4 (default, Apr  9 2021, 09:32:38) 
[Clang 10.0.0 ]
numpy version = 1.19.5
tensorflow version = 2.5.0


## Define experiment settings and directories

In [5]:
EXP_NAME = 'quadrants_testcase'

imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

imp.reload(common_functions)
model_dir, model_diagnostics_dir, vizualization_dir = common_functions.get_exp_directories(EXP_NAME)

## Define the network parameters

In [6]:
RANDOM_SEED          = settings['random_seed']
BATCH_SIZE_PREDICT   = settings['batch_size_predict']
BATCH_SIZE           = settings['batch_size']
NLAYERS              = settings['nlayers']
NFILTERS             = settings['nfilters']   
assert(len(NFILTERS)==NLAYERS)

NCLASSES             = settings['nclasses']
PROTOTYPES_PER_CLASS = settings['prototypes_per_class']
NPROTOTYPES          = np.sum(PROTOTYPES_PER_CLASS)

NEPOCHS              = settings['nepochs']
LR_INIT              = settings['lr']
LR_CALLBACK_EPOCH    = settings['lr_cb_epoch']
PATIENCE             = 100

## Initialize

In [7]:
np.random.seed(RANDOM_SEED)
rng = np.random.default_rng(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## Get and process the data

In [8]:
imp.reload(data_functions)
DATA_NAME = settings['data_name']
DATA_DIR = settings['data_dir']

if(EXP_NAME[:3]=='mjo'):

    labels, data, lat, lon, time = data_functions.load_mjo_data(DATA_DIR)
    X_train, y_train, time_train, X_val, y_val, time_val, X_test, y_test, time_test = data_functions.get_and_process_mjo_data(labels,
                                                                                         data,
                                                                                         time,
                                                                                         rng, 
                                                                                         colored=settings['colored'],
                                                                                         standardize=settings['standardize'],
                                                                                         shuffle=settings['shuffle'],
                                                                                        )        
elif(EXP_NAME[:9]=='quadrants'):
    filename = DATA_DIR + DATA_NAME + '.mat'
    X_train, y_train, X_val, y_val, X_test, y_test, lat, lon = data_functions.get_and_process_data(filename, 
                                                                                        rng, 
                                                                                        colored=settings['colored'],
                                                                                        standardize=settings['standardize'],
                                                                                        shuffle=settings['shuffle'],
                                                                                        )      
    

loading ./data/data_quadrants_testcase.mat
shuffling the data before train/validation/test split.
raw_data        = (1200, 100, 100, 1)
training data   = (960, 100, 100, 1), (960,)
validation data = (240, 100, 100, 1), (240,)
test data       = (0, 100, 100, 1), (0,)
X_mean          = 0.0008956050790393463
X_std           = 0.11342437932997024


In [9]:
proto_class_mask = network.createClassIdentity(PROTOTYPES_PER_CLASS)

prototypes_of_correct_class_train = np.zeros((len(y_train),NPROTOTYPES))
for i in range(0,prototypes_of_correct_class_train.shape[0]):
    prototypes_of_correct_class_train[i,:] = proto_class_mask[:,int(y_train[i])]
    
prototypes_of_correct_class_val   = np.zeros((len(y_val),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_val.shape[0]):
    prototypes_of_correct_class_val[i,:] = proto_class_mask[:,int(y_val[i])]

prototypes_of_correct_class_test   = np.zeros((len(y_test),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_test.shape[0]):
    prototypes_of_correct_class_test[i,:] = proto_class_mask[:,int(y_test[i])]
    

## Define the training callbacks and metrics

In [10]:
# callbacks
def scheduler(epoch, lr):
    if epoch < LR_CALLBACK_EPOCH:
        return np.round(lr,8)
    else:
        if(epoch % 2 == 0):
            return lr/2.
        else:
            return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)    
    
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    mode='max',
    patience=settings['patience'], 
    restore_best_weights=True, 
    verbose=1
)

callbacks_list = [
#     lr_callback,
#     es_callback,
]            

# metrics
metrics_list = [
    tf.keras.metrics.SparseCategoricalAccuracy(),
]

## Instantiate the model

In [11]:
__ = imp.reload(network)
tf.keras.backend.clear_session()

model = network.build_model(
    nlayers              = NLAYERS,
    nfilters             = NFILTERS,
    input_shape          = X_train.shape[1:],
    output_shape         = NCLASSES,
    prototypes_per_class = PROTOTYPES_PER_CLASS,
    network_seed         = RANDOM_SEED,    
    prototype_channels   = settings['prototype_channels'],    
    coeff_cluster        = settings['coeff_cluster'],
    coeff_separation     = settings['coeff_separation'],
    coeff_l1             = settings['coeff_l1'],
    incorrect_strength   = settings['incorrect_strength'],
    double_conv          = settings['double_conv'],
    kernel_l1_coeff      = 0.0,#settings['kernel_l1_coeff'],
    kernel_l2_coeff      = 0.0,#settings['kernel_l2_coeff'],
    drop_rate            = 0.0,
    drop_rate_final      = 0.0,        
    
)
model.summary()

2
[32, 32]
(100, 100, 1)
3
[5, 5, 5]
35
False
0.17
-0.017
0.5
-0.5
False
8
128
0.0
0.0
0.0
0.0
False
Model: "full_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 320         inputs[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 100, 100, 32) 0           conv_0[0][0]                     
__________________________________________________________________________________________________
maxpooling_0 (AveragePooling2D) (None, 50, 50, 32)   0           dropout[0][0]         

## Load pre-trained weights into convolutional layers

In [12]:
if(settings['pretrain'] == True):

    if(settings['pretrain_exp'] is None):
        PRETRAINED_MODEL = model_dir + 'pretrained_model_' + EXP_NAME 
    else:
        PRETRAINED_MODEL = './saved_models/' + settings['pretrain_exp'] 

    print('loading pretrained convolutional layers from ' + PRETRAINED_MODEL)
    pretrained_model = tf.keras.models.load_model(PRETRAINED_MODEL)

    for layer in range(1,len(model.layers)):
        if(model.layers[layer].name[:4]=='conv'):
            print('   loading pretrained weights for --> ' + model.layers[layer].name)
            model.layers[layer].set_weights(pretrained_model.layers[layer].get_weights())
else:
    print('no pretrained model specified. keeping random initialized weights.')
    

loading pretrained convolutional layers from ./saved_models/quadrants_testcase/pretrained_model_quadrants_testcase
   loading pretrained weights for --> conv_0
   loading pretrained weights for --> conv_1


In [13]:
# raise ValueError('here')

***

# Run Training Stages

In [14]:
imp.reload(network)
imp.reload(plots)
imp.reload(push_prototypes)
imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

In [15]:
ic(np.shape(X_train))
ic(np.shape(prototypes_of_correct_class_train))
ic(np.shape(prototypes_of_correct_class_train))

ic| np.shape(X_train): (960, 100, 100, 1)
ic| np.shape(prototypes_of_correct_class_train): (960, 15)
ic| np.shape(prototypes_of_correct_class_train): (960, 15)


(960, 15)

In [16]:
imp.reload(push_prototypes)
NEPOCHS    = settings['nepochs']
STAGE_LIST = (0,1,2,3,4,5,6,7,8,9)#range(len(NEPOCHS))#(1,2,3,4,5)#range(len(NEPOCHS))

for stage in STAGE_LIST:
    
    print('--------------------')
    print('TRAINING STAGE = ' + str(stage))
    print('--------------------')

    # load previously trained stage, unless it is the 0th stage
    if(stage != 0):
        tf.keras.backend.clear_session()
        model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage-1)
#         model = common_functions.load_model(model_filename)
        model.load_weights(model_filename)
        
    # learn layers (during even numbered stages)
    if(stage % 2 == 0):
        # train prototypes layers (and possibly CNN layers)
        if(settings['pretrain']==False and settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(settings['train_cnn_in_stage'] == False or stage==0):
            model = network.set_trainable_layers(model, [False,True,True,False])
        elif(settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(stage >= settings['train_cnn_in_stage']):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        else:
            model = network.set_trainable_layers(model, [False,True,True,False])
    else:
        #.......................................................
        # push the prototypes
        #.......................................................        
        model, push_info = push_prototypes.push(model, 
                                                [X_train,prototypes_of_correct_class_train], 
                                                prototypes_of_correct_class_train, 
                                                perform_push=True,
                                                batch_size=BATCH_SIZE_PREDICT,
                                                verbose=False,
                                               )        
        print('Push complete.\n')            

        # train weights layer only
        model = network.set_trainable_layers(model, [False,False,False,True])        

    #.......................................................
    # compile the model
    #.......................................................
    if(stage>=settings['cut_lr_stage']):
        lr_factor = 10.**(np.floor((stage-settings['cut_lr_stage']+2)/2))
    else:
        lr_factor = 1.
    if(LR_INIT/lr_factor<settings['min_lr']):
        lr_factor = LR_INIT/settings['min_lr']
    print('learning rate = ' + str(np.asarray(LR_INIT/lr_factor,dtype='float32')))

    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=np.asarray(LR_INIT/lr_factor,dtype='float32'), 
        ),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics = metrics_list,
    )
#     model.summary()
    ic(np.min(model.layers[-3].get_weights()[1]),np.max(model.layers[-3].get_weights()[1]))

    #.......................................................
    # train the model
    #.......................................................
    print('Training the model...')    
    
    tf.random.set_seed(RANDOM_SEED)   
    np.random.seed(RANDOM_SEED)    
    history = model.fit(
        [X_train,prototypes_of_correct_class_train],
        y_train,
        validation_data=([[X_val,prototypes_of_correct_class_val]], [y_val]),
        batch_size=BATCH_SIZE,
        epochs=NEPOCHS[stage],
        shuffle=True,
        verbose=1,
        callbacks=callbacks_list
    )
    print('Training complete.\n')            
        

    # save the model at this training stage
    model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage)
    common_functions.save_model(model, model_filename) 
    
    #.......................................................
    # plot results
    #.......................................................  
    try:
        # plot loss history of the model
        plots.plot_loss_history(history)
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_loss_history_stage' + str(stage) + '.png', dpi=dpiFig)    
        plt.close()

        # plot the weights
        plots.plot_weights(model, PROTOTYPES_PER_CLASS)    
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_weights_stage' + str(stage) + '.png', dpi=dpiFig)
        plt.close()
    except:
        print('not making plots...')
        plt.close()

ic| np.min(model.layers[-3].get_weights()[1]): 0.0
    np.max(model.layers[-3].get_weights()[1]): 0.0


--------------------
TRAINING STAGE = 0
--------------------
   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 0.01
Training the model...
Epoch 1/10
30/30 [==============================] - 3s 66ms/step - loss: 9.8475 - sparse_categorical_accuracy: 0.3417 - cluster_cost: 8.1030 - separation_cost: 7.5587 - l1_weights_cost: 7.5000 - val_loss: 9.0805 - val_sparse_categorical_accuracy: 0.3208 - val_cluster_cost: 3.0878 - val_separation_cost: 2.6834 - val_l1_weights_cost: 7.5000
Epoch 2/10
30/30 [==============================] - 2s 58ms/step - loss: 8.8842 - sparse_categorical_accuracy: 0.3812 - cluster_cost: 1.9063 - separation_cost: 1.7261 - l1_weights_cost: 7.5000 - val_loss: 8.7693 - val_sparse_categorical_accuracy: 0.3458 - val_cluster_cost: 1.1116 - val_separation_cost: 0.9538 - val_l1_weights_cost: 7.5000
Epoch 3/10
30/30 [

ic| np.min(model.layers[-3].get_weights()[1]): -0.46120986
    np.max(model.layers[-3].get_weights()[1]): 2.6305072


Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 0.01
Training the model...
Epoch 1/10
30/30 [==============================] - 2s 40ms/step - loss: 6.5363 - sparse_categorical_accuracy: 0.7467 - cluster_cost: 0.0482 - separation_cost: 0.0402 - l1_weights_cost: 5.4331 - val_loss: 4.4521 - val_sparse_categorical_accuracy: 0.6917 - val_cluster_cost: 0.0470 - val_separation_cost: 0.0359 - val_l1_weights_cost: 3.2381
Epoch 2/10
30/30 [==============================] - 1s 34ms/step - loss: 2.2338 - sparse_categorical_accuracy: 0.7344 - cluster_cost: 0.0482 - separation_cost: 0.0402 - l1_weights_cost: 1.3596 - val_loss: 1.3813 - val_sparse_categorical_accuracy: 0.6833 - val_cluster_cost: 0.0470 - val_separation_cost: 0.0359 - val_l1_weights_cost: 0.4047
Epoch 3/10
30/30 [============

ic| np.min(model.layers[-3].get_weights()[1]): -0.46120986
    np.max(model.layers[-3].get_weights()[1]): 2.6305072


...

Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 0.001
Training the model...
Epoch 1/10
30/30 [==============================] - 2s 38ms/step - loss: 0.1326 - sparse_categorical_accuracy: 0.9717 - cluster_cost: 1.8616e-04 - separation_cost: 4.1707e-05 - l1_weights_cost: 0.0479 - val_loss: 0.2792 - val_sparse_categorical_accuracy: 0.9500 - val_cluster_cost: 2.3651e-04 - val_separation_cost: 4.9591e-05 - val_l1_weights_cost: 0.0464
Epoch 2/10
30/30 [==============================] - 1s 32ms/step - loss: 0.1250 - sparse_categorical_accuracy: 0.9719 - cluster_cost: 1.8616e-04 - separation_cost: 4.1707e-05 - l1_weights_cost: 0.0430 - val_loss: 0.2726 - val_sparse_categorical_accuracy: 0.9500 - val_cluster_cost: 2.3651e-04 - val_separation_cost: 4.9591e-05 - val_l1_weights_cost: 0.04

ic| np.min(model.layers[-3].get_weights()[1]): -0.6448027
    np.max(model.layers[-3].get_weights()[1]): 3.5722928


--------------------
TRAINING STAGE = 8
--------------------
   conv_0 --> True
   maxpooling_0 --> True
   conv_1 --> True
   maxpooling_1 --> True
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 1e-04
Training the model...
Epoch 1/10
30/30 [==============================] - 4s 108ms/step - loss: 0.1161 - sparse_categorical_accuracy: 0.9708 - cluster_cost: 3.1230e-04 - separation_cost: 9.7656e-05 - l1_weights_cost: 0.0411 - val_loss: 0.2648 - val_sparse_categorical_accuracy: 0.9458 - val_cluster_cost: 4.6921e-04 - val_separation_cost: 2.1744e-04 - val_l1_weights_cost: 0.0411
Epoch 2/10
30/30 [==============================] - 3s 104ms/step - loss: 0.1056 - sparse_categorical_accuracy: 0.9812 - cluster_cost: 7.5073e-04 - separation_cost: 6.2663e-04 - l1_weights_cost: 0.0411 - val_loss: 0.2589 - val_sparse_categorical_accuracy: 0.9417 - val_cluster_cost: 9.1934e-04 - val_separation_cost: 8.8501e-04 - val_l1_weights_

ic| np.min(model.layers[-3].get_weights()[1]): -0.6448027
    np.max(model.layers[-3].get_weights()[1]): 3.576061


Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 1e-04
Training the model...
Epoch 1/10
30/30 [==============================] - 2s 39ms/step - loss: 0.1055 - sparse_categorical_accuracy: 0.9758 - cluster_cost: 1.3326e-04 - separation_cost: 2.1362e-05 - l1_weights_cost: 0.0368 - val_loss: 0.2729 - val_sparse_categorical_accuracy: 0.9417 - val_cluster_cost: 1.2589e-04 - val_separation_cost: 1.9073e-05 - val_l1_weights_cost: 0.0353
Epoch 2/10
30/30 [==============================] - 1s 33ms/step - loss: 0.1033 - sparse_categorical_accuracy: 0.9812 - cluster_cost: 1.3326e-04 - separation_cost: 2.1362e-05 - l1_weights_cost: 0.0349 - val_loss: 0.2724 - val_sparse_categorical_accuracy: 0.9417 - val_cluster_cost: 1.2589e-04 - val_separation_cost: 1.9073e-05 - val_l1_weights_cost: 0.03